In [ ]:
#Import the necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models

# ***Loading Dataset***

In [ ]:
import gdown
import zipfile

url = 'https://drive.google.com/uc?id=1l9sYu1jnRvFwTXte77mGew0fvopKYY3n'
output = 'dataset.zip'

gdown.download(url, output, quiet=False)

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('extracted_dataset')

Downloading...
From: https://drive.google.com/uc?id=1l9sYu1jnRvFwTXte77mGew0fvopKYY3n
To: /content/dataset.zip
100%|██████████| 3.03G/3.03G [00:30<00:00, 101MB/s]


In [ ]:
#Define Dataset paths & Transformations
train_dir = '/content/extracted_dataset/train'
test_dir = '/content/extracted_dataset/test'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
#Load the training and testing datasets
train_dataset = ImageFolder(train_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

In [ ]:
#Create data loaders for training and validation
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## ***VGG 16 Model***

In [ ]:
#Define VGG16 Model
model = models.vgg16()

In [ ]:
#Modify Model's Last FC Layer
num_classes = len(train_dataset.classes)
print("Number of Classes: ",num_classes)
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, num_classes)

Number of Classes:  14


In [ ]:
#Define Loss function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
#Train VGG16 Model & Calculating Accuaracy of Model
num_epochs = 4

for epoch in range(num_epochs):
    # Training loop
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)

    # Calculate average losses and accuracy
    train_loss = train_loss / len(train_dataset)
    val_loss = val_loss / len(test_dataset)
    val_accuracy = val_correct.double() / len(test_dataset)

    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.4f}")